## Local Fucntion Review


In [4]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
    return inner_function()

In [5]:
outer_function()

Hi


In [15]:
def outer_function(msg):
    message = msg
    def inner_function():
        print(message)
        
    #return the fucntion object
    return inner_function

In [19]:
my_func=outer_function("hi")
bye_func=outer_function("bye")

In [21]:
my_func()
bye_func()

hi
bye


In [23]:
def outer_function(msg):
    def inner_function():
        print(msg)
        
    #return the fucntion object
    return inner_function

In [24]:
my_func=outer_function("hi")
bye_func=outer_function("bye")
my_func()
bye_func()

hi
bye


## Decorators
- A fucntion that takes another function as an arguement
- Adds some functionality
- Returns another function
- All of this without altering the original function

In [28]:
def decorator_function(original_function):
    def wrapper_function():
        return original_function()
    #return function object
    return wrapper_function

def display():
    print("The display function ran.")

In [32]:
#test it
decorated_display = decorator_function(display)
print(decorated_display)
print(type(decorated_display))
decorated_display()

<function decorator_function.<locals>.wrapper_function at 0x000002141A0671E0>
<class 'function'>
The display function ran.


## Make it more interesting

In [42]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrappper executed this before {}.".format(original_function.__name__))
        return original_function()
    #return function object
    return wrapper_function

def display():
    print("The display function ran.")

In [43]:
#test it
decorated_display = decorator_function(display)
print(decorated_display)
print(type(decorated_display))
decorated_display()

<function decorator_function.<locals>.wrapper_function at 0x0000021419FDB620>
<class 'function'>
Wrappper executed this before display.
The display function ran.


## More decorations
use the @Decorator_function_name

In [45]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrappper executed this before {}.".format(original_function.__name__))
        return original_function()
    #return function object
    return wrapper_function

@decorator_function
def display():
    print("The display function ran.")

In [46]:
#test it
display()

Wrappper executed this before display.
The display function ran.


This will not work if the original function takes arguements

In [51]:
@decorator_function
def display_info(name, age):
    print("display ran with arguements({}, {})".format(name, age))


In [52]:
display_info("Mario", 21)

TypeError: wrapper_function() takes 0 positional arguments but 2 were given

## Need to fix the wrapper.
We can fix this with **\*args** and **\*\*kwargs**.

In [53]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print("Wrappper executed this before {}.".format(original_function.__name__))
        return original_function(*args, **kwargs)
    #return function object
    return wrapper_function

@decorator_function
def display():
    print("The display function ran.")
    
@decorator_function
def display_info(name, age):
    print("display ran with arguements({}, {})".format(name, age))

In [56]:
display()
print()
display_info("Mario", 21)

Wrappper executed this before display.
The display function ran.

Wrappper executed this before display_info.
display ran with arguements(Mario, 21)


## Task:
Define a decortor escape unicode
Convert to ascii and return the equivilant.


In [70]:
def escape_unicode(f):
    def wrapper(*args, **kwargs):
        print("Wrappper executed this before {}.".format(f.__name__))
        x = f(*args, **kwargs)
        return ascii(x)
    return wrapper

@escape_unicode
def mexico_city():
    #alt 130 for é.
    return "México City" 

In [71]:
mexico_city()

Wrappper executed this before mexico_city.


"'M\\xe9xico City'"

#### What can be a decorator?
- Class object, which can be callable with the dunder call.
- Functions as decorators

### Classes as decorators

In [75]:
class DecoratorClass(object):
    def __init__(self, original_function):
        self.original_function = original_function
        
    def __call__(self, *args, **kwargs):
        print("Call method executed this before {}.".format(self.original_function))
        return self.original_function(*args, **kwargs)
    
@DecoratorClass
def display():
    print("The display function ran.")
    
@DecoratorClass
def display_info(name, age):
    print("display ran with arguements({}, {})".format(name, age))

In [76]:
display()
print()
display_info("Mario", 21)

Call method executed this before <function display at 0x000002141A057EA0>.
The display function ran.

Call method executed this before <function display_info at 0x000002141A057F28>.
display ran with arguements(Mario, 21)


In [89]:
class Trace:
    def __init__(self):
        self._enable = True
        
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self._enable:
                print("Calling {}.".format(f.__name__))
            return f(*args, **kwargs)
        return wrap
    
tracer = Trace()

@tracer
def rotate_list(l):
    return l[1:]+ [l[0]]

Unlike our previous example, the **class object its self** is not the decorator. The instance of trace here is the decorator. 

In [91]:
l1 = [1, 2, 3, 4]
l1 = rotate_list(l1)

Calling rotate_list.


In [92]:
l1

[2, 3, 4, 1]

In [127]:
class Trace:
    def __init__(self):
        self._enable = True
        
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self._enable:
                print("Calling {}.".format(f.__name__))
            return f(*args, **kwargs)
        return wrap
    
tracer2 = Trace()

@tracer2
def rotate_list(l):
    return l[1:]+ [l[0]]

tracer2._enable = False
l1 = [1, 2, 3, 4]
l2 = rotate_list(l1)

In [139]:
import time

def my_timer(original_func):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_func(*args, **kwargs)
        t2 = time.time() - t1
        print ("{} ran in: {}".format(original_func.__name__, t2))
        return result
    return wrapper

@my_timer
def display_info(name, age):
    time.sleep(1)
    print("display ran with arguements({}, {})".format(name, age))
    

In [140]:
display_info("Mario", 21)

display ran with arguements(Mario, 21)
display_info ran in: 1.0088765621185303


## Multiple Decorators

In [138]:
def my_logger(original_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(original_func.__name__), level=logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}, and kwargs {}.".format(args, kwargs))
        return original_func(*args, *kwargs)
    return wrapper
    
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("display ran with arguements({}, {})".format(name, age))
    

In [120]:
display_info("Mario", 21)

display ran with arguements(Mario, 21)


In [141]:
#Now with multiple decorators
@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("display ran with arguements({}, {})".format(name, age))

#test it
display_info("Mario", 21)

display ran with arguements(Mario, 21)
wrapper ran in: 1.0002362728118896


In [142]:
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("display ran with arguements({}, {})".format(name, age))

#test it
display_info("Mario", 21)

display ran with arguements(Mario, 21)
display_info ran in: 1.0003445148468018


Wrap everything

In [131]:
import time
from functools import wraps

def my_logger(original_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(original_func.__name__), level=logging.INFO)
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}, and kwargs {}.".format(args, kwargs))
        return original_func(*args, *kwargs)
    return wrapper

def my_timer(original_func):
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_func(*args, **kwargs)
        t2 = time.time() - t1
        print ("{} ran in: {}".format(original_func.__name__, t2))
        return result
    return wrapper

In [135]:
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("display ran with arguements({}, {})".format(name, age))

#test it
display_info("Mario", 21)

display ran with arguements(Mario, 21)
display_info ran in: 1.0001890659332275


In [134]:
@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("display ran with arguements({}, {})".format(name, age))

#test it
display_info("Mario", 21)

display ran with arguements(Mario, 21)
display_info ran in: 1.000190019607544
